In [1]:
import numpy as np
import time
from copy import deepcopy as dc

In [2]:
k0 = 79.5 #[W/m K] thermal conductivity of Fe
rho_Fe = 7870. #[kg/m^3] density of kamacite (like Elkins-Tanton+ 2020 used)
Cp = 850 #[J/kg K] Heat capacity (of Iron)
alpha = k0 /(rho_Fe*Cp) #[m^2/s] Thermal diffusivity of Fe
G = 6.67e-11 #[m^3/kg– s^2] Big G

In [12]:
def pressure(rs, phis, iron=True):
    dms = np.zeros(len(rs))
    gs = np.zeros(len(rs))
    Ps = np.zeros(len(rs))
    
    if iron:
        rho = rho_Fe
    else:
        rho = 3500 #Bierson & Nimmo (2019) rocky density
    
    dms[1:] = 4/3*np.pi*rho*(1-phis[0:-1])*(rs[1:]**3-rs[0:-1]**3)
    for i in range(1,len(rs)):
        gs[i] = G/rs[i]**2 * np.sum(dms[0:i+1])
    for i in range(len(rs)-2,-1,-1):
        dr = rs[i+1]-rs[i]
        Ps[i] = Ps[i+1] + rho*(1-phis[i])*gs[i]*dr
    return Ps

def visc(T, iron=True):
    if iron:
        n0 = 1e11 
        T0 = 1800 
        Q = 2.5e5
        R = 8.3145
        
        return n0*np.exp(Q/(R*T)-Q/(R*T0)) 
        
    else: #diffusion creep in dry olivine (Karato & Wu, 1993)
        n0 = 4.05e11
        Q = 3e5
        R = 8.3145
        return n0*np.exp(Q/(R*T))


def get_alpha(phi, iron=True):
    if iron:
        k = k0*(1-phi)
        return k / (rho_Fe*Cp)
    else: #Beirson & Nimmo (2019) vals
        krock = 3.0 
        rho_rock = 3500 
        Cp_rock = 1000 
        k = krock*(1-phi)
        return k / (rho_rock * Cp_rock)

def dphi(xs, T, phi, iron=True):
    n = visc(T, iron)
    P = pressure(xs,phi,iron)
    return phi*(P/n)

def porosity_update(xs, Ts, phi0, dt, iron=True):    
    dp = dphi(xs, Ts, phi0, iron)*dt
    phi = phi0 - dp
    
    phi[np.where(phi < 0)] = 0.
    
    a = get_alpha(phi, iron)
    return a, phi

##############################################################################################################

def velocity(n, k): #not allowing right now
    return np.asarray([0]*len(n)) 

def source(typ, n, k): #just for iron right now
    rho_H0 = 3.7e-6 #[W/m^3] - random value for now
    S0 = rho_H0 / (rho_Fe * Cp)
    if typ == 'linear':
        r = n*dx
        R = N*dx
        return S0 / R * r #(N*dx - n*dx)
    elif typ == 'constant':
        return np.asarray([S0]*len(n))
    else:
        return np.asarray([0]*len(n))
    
def next_step_s(alpha, aa, ab, Tnk, vnk, Snk, Tbelow, Tabove, r):
    return Tnk*(1 - 2*vnk*dt/r) + alpha * (dt / dx**2) * (Tabove - 2*Tnk + Tbelow) + (dt / (2*dx)) * \
                (2*alpha/r - vnk + ((aa-ab)/(2*dx))) * (Tabove - Tbelow) + dt * Snk

def Nstep_s(alpha, C, fluxBC, Tnk, vnk, Snk, Tbelow, R):
    if fluxBC: #need a term for da/dr, but not using this type of BC so it's ok
        return Tnk*(1 - 2*vnk*dt/R) + 2 * alpha * (dt / dx**2) * (Tbelow + C*dx - Tnk) + (2*alpha/R - vnk) * \
                dt * C + dt * Snk
    else:
        return C

def main(xs, IC, BC1, BC2, phi0, tmax, fluxBC=False, fluxBC2=True, s_typ=None, iron=True):
    if not (np.shape(xs) == np.shape(IC)):
        print('Error: Initial Conditions Shape Mismatch')
        return
    Tinit = IC
    Ts = [Tinit]
    
    alpha = get_alpha(phi0, iron)
    phis = [phi0]
    shape = [xs]
    time = [0]
    limit = np.nanmin(np.diff(xs)**2/(2*alpha[:-1]))
    dt = limit/10
    M = 1e8
    t = dt
    
    #spherical coordinates
    for k in range(1,int(M)+1): #timesteps
        a = xs[0]
        dxs = np.diff(xs)
        Tk = Ts[k-1]
        Tnext = np.zeros(np.shape(Tk))
        vk = velocity(np.arange(len(Tnext)), k)
        Sk = source(s_typ, np.arange(len(Tnext)), k)

        limits = dxs**2 / (2*alpha[:-1])
        if np.nanmin(limits) < dt:
            print('Unstable timestep')
            return np.asarray(phis), np.asarray(Ts), np.asarray(shape), np.asarray(time)
        #deal with x = a
        if not fluxBC: #constant T(0)
            if a == 0:
                print('Error: Must have 0 flux at r=0')
                return
            else:
                Tnext[0] = BC1

        else: #zero flux BC at x=a - NO SOURCE ALLOWED!
            Tnext[0] = Tk[0] + 2*alpha[0] * (dt / dxs[0]**2) * (Tk[1] - BC1*dxs[0] - Tk[0])                  

        Tnext[1:-1] = Tk[1:-1]*(1-2*vk[1:-1]*dt/xs[1:-1]) + alpha[1:-1]*(dt/dxs[1:]**2)*(Tk[2:]-2*Tk[1:-1] + \
                            Tk[0:-2])+(dt/(2*dxs[1:]))*(2*alpha[1:-1]/xs[1:-1]-vk[1:-1] + \
                            ((alpha[2:]-alpha[0:-2])/(2*dxs[1:]))) * (Tk[2:] - Tk[0:-2]) + dt * Sk[1:-1]

        Tnext[-1] = Nstep_s(alpha[-1], BC2, fluxBC2, Tk[-1], vk[-1], Sk[-1], Tk[-2], xs[-1])

        alpha, new = porosity_update(xs, Tk, phi0, dt, iron)

        rs = dc(xs)    
        phi = dc(phi0)

        for i in range(len(phi0)):
            dxs = np.diff(rs)
            phi0 = dc(phi)
            phi = dc(phi0)
            phi[i] = new[i]

            new_rs = [0]
            psi = (1-phi0[0]) / (1-phi[0])
            Rt0 = rs[1]
            db = (np.cbrt((1-(dxs[0]/Rt0))**3*(1-psi)+psi) - 1)*Rt0
            Rtf = Rt0 + db
            new_rs.append(Rtf)
            for j in range(2,N+1):
                da = db
                Rt0 = rs[j]
                dr = dxs[j-1]
                psi = (1-phi0[j-1])/(1-phi[j-1])
                db1 = (np.cbrt((1+(da-dr)/Rt0)**3-(1-dr/Rt0)**3+1)-1)*Rt0 
                db2 = (np.cbrt((1-(dr/Rt0))**3*(1-psi)+psi) - 1)*Rt0
                db = db1+db2
                Rtf = Rt0 + db
                new_rs.append(Rtf)

            rs_v2 = np.asarray(new_rs)
            rs = dc(rs_v2)

        Ts.append(Tnext)
        phis.append(phi)
        shape.append(rs)
        time.append(t)

        change = np.nanmax(abs(phis[k]-phis[k-1]))

        #change resolution if necessary
        limits = np.diff(rs)**2 / (2*alpha[:-1])
        dt = np.nanmin(limits) / 10 #take 1/10th of stability limit as timestep

        xs = dc(rs) #update to conserve mass
        phi0 = dc(phi)
        t += dt

        if t > tmax*3.154e+7:
            print('Reached tmax')
            return np.asarray(phis), np.asarray(Ts), np.asarray(shape), np.asarray(time)
    
    return np.asarray(phis), np.asarray(Ts), np.asarray(shape), np.asarray(time)

# Example Psyche Model

In [13]:
phi0 = 0.7 #initial porosity
Ti = 600 #initial temperature
M = 2.29e19 #mass of body
tmax = 1e5 #in seconds

xmax = int(np.round(np.cbrt(M / (4/3*np.pi*rho_Fe*(1-phi0)))/1e3)) #in km, matches measured mass

N = xmax #want 1 km res
dx = (xmax * 1e3) / N

In [14]:
T0 = np.full((N+1), Ti)
T0[-1] = 137
start = time.time()
ps, psyche, shape, times = main(np.arange(0,xmax*1e3+dx, dx), T0, 0, 137, np.full((N+1),phi0), tmax, fluxBC=True, 
                    fluxBC2=False, s_typ=None, iron=True)
print("--- %s minutes ---" % ((time.time() - start)/60))

Reached tmax
--- 0.45878366629282635 minutes ---


In [462]:
full = {'times':times,
         'temps': psyche,
         'porosity': ps,
         'shapes': shape
        }

np.save('PsycheMass70percent600K.npy', full)